# GiveMeCredit Kaggle Submission


----

#### *Chargement du Dataset Full Give Me Credit - Kaggle*

In [1]:
from bigml.api import BigML
from pandas import read_csv
from ipynb.fs.defs.fonctions import features

df = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-train.csv', index_col=0)

On stocke le train full

In [2]:
%store df

Stored 'df' (DataFrame)


#### *Connexion BigML*

In [3]:
api = BigML(project="project/5d94a3525a213962e20002f5") # AUTH dans docker/auth.env

#### *Modifications du dataset Trainfull*

In [4]:
df = features(df)
df.to_csv('files_csv/origin_dataset_modif.csv')
print("Load & Modifications OK")

Load & Modifications OK


#### *Création sur BIGML source -> dataset*

In [5]:
# Creation d'une source
source = api.create_source('files_csv/origin_dataset_modif.csv')
api.ok(source)
# Creation d'un dataset ( = source )
origin_dataset = api.create_dataset(source)
api.ok(origin_dataset)

You don't have enough credits available: [{'credit_size': 18025114, 'credit_type': 300}]
You'll need to buy some more credits to perform the chosen action


2019-12-07 10:31:27,292: You don't have enough credits available: [{'credit_size': 18025114, 'credit_type': 300}]
You'll need to buy some more credits to perform the chosen action


The resource couldn't be created: {'code': 402, 'status': {'code': -1101, 'extra': [{'credit_size': 18025114, 'credit_type': 300}], 'message': "You don't have enough credits available"}}


2019-12-07 10:31:27,293: The resource couldn't be created: {'code': 402, 'status': {'code': -1101, 'extra': [{'credit_size': 18025114, 'credit_type': 300}], 'message': "You don't have enough credits available"}}


#### *Split du trainfull en Train/Test*

In [6]:
train_dataset = api.create_dataset(
    origin_dataset, {"name": "GiveMeCredit | TrainFull | Training",
                     "sample_rate": 0.8, "seed": "my seed"})
test_dataset = api.create_dataset(
    origin_dataset, {"name": "GiveMeCredit | TrainFull | Test",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})
print("Split OK")

ValueError: Failed to parse a resource string or structure.

#### *Création d'un modele ensemble sur la partie Train du dataset ( objective_field = ce qu'on cherche à prédire)*

In [ ]:
ensemble = api.create_ensemble(train_dataset , {"objective_field" : "SeriousDlqin2yrs"})
print("Création model OK")

#### *Création et téléchargement de l'évaluation de notre modèle*

In [ ]:
evaluation = api.create_evaluation(ensemble, test_dataset)
api.export(evaluation, filename="EvaluationModel/my_evaluation_model.json")

#### *Verification du modele sur les 20% du Train Full et téléchargement*


In [ ]:
batch_prediction = api.create_batch_prediction(ensemble, test_dataset,{"header": True, "all_fields": True, "probabilities": True})
api.ok(batch_prediction)
api.download_batch_prediction(batch_prediction,filename='files_csv/GiveMeCredit_Review_Ensemble.csv')


#### *Modifications fichier test kaggle nouvelles features*

In [7]:
df_test_kaggle = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-test.csv', index_col=0)
%store df_test_kaggle
df_test_kaggle = features(df_test_kaggle)
df_test_kaggle.to_csv('files_csv/kaggle_dataset_modif.csv')
print("Load & Modifications OK")

Stored 'df_test_kaggle' (DataFrame)
Load & Modifications OK


#### *Création d'une source avec le fichier modifié test kaggle*

In [ ]:
source_kaggle = api.create_source('files_csv/kaggle_dataset_modif.csv')
api.ok(source_kaggle)

#### *Création d'un dataset à partir de la source*

In [ ]:
kaggle_dataset= api.create_dataset(source_kaggle)
api.ok(kaggle_dataset)

#### *Prédiction de notre modèle sur le fichier test de Kaggle et téléchargement*

In [ ]:
batch_prediction_kaggle = api.create_batch_prediction(ensemble, kaggle_dataset,{"all_fields": True,"probabilities": True})
api.ok(batch_prediction_kaggle)
api.download_batch_prediction(batch_prediction_kaggle,filename='files_csv/GiveMeCredit_Prediction_Kaggle.csv')

#### *Formatage du fichier de prédiction au format attendu par Kaggle*

In [ ]:
from pandas import read_csv

df_final_prediction = read_csv("files_csv/GiveMeCredit_Prediction_Kaggle.csv")
keep_col = ['Id','1 probability']
new_final_prediction = df_final_prediction[keep_col]
new_final_prediction.rename(columns={'1 probability':'Probability'}, inplace=True)
new_final_prediction.to_csv('files_csv/GiveMeCredit_Kaggle_format.csv', index=False)
print("Modifications OK")

#### *Envoi de nos résultats à Kaggle*

In [ ]:
# import kaggle
# submission_file = 'files_csv/GiveMeCredit_Kaggle_format.csv'
# kaggle.api.competition_submit(submission_file, "BigML ensemble", "GiveMeSomeCredit")